In [36]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
import sys
from scipy.optimize import brentq, least_squares
from scipy.stats import norm
from scipy.interpolate import RegularGridInterpolator

sys.path.append("..")

# handy it seems
# https://docs.sympy.org/latest/modules/solvers/solvers.html
from sympy.solvers import solve
from sympy import Symbol
from analytical_option_formulae.option_types.vanilla_option import VanillaOption


# read data
swaption_data = pd.read_csv("../data/Swaption_Data.csv")
df_snN_0 = pd.read_csv("../bootstrap_swap_curve/df_1c.csv")
df_combined = pd.read_csv("../bootstrap_swap_curve/df_combined.csv")

# use dict comprehension?
tenor_mapping = {
    "6m": 0.5,
    "1Y": 1.0,
    "2Y": 2.0,
    "3Y": 3.0,
    "5Y": 5.0,
    "10Y": 10.0,
}

# swaption_data processing
swaption_data["Expiry"] = swaption_data["Expiry"].map(tenor_mapping)
swaption_data["Tenor"] = swaption_data["Tenor"].map(tenor_mapping)
swaption_data[swaption_data.columns[2:]] = (
    swaption_data[swaption_data.columns[2:]] / 100
)
swaption_data.columns = swaption_data.columns.str.lower()
# df_snN_0 processing
df_snN_0.rename(
    columns={
        "maturity": "expiry",
        "duration": "tenor",
        "pv_fix_nok": "pvbp",
        "k_rate": "snN_0",
    },
    inplace=True,
)

# DAY COUNT CONVENTION IS 30/360
FULL_YEAR = 360

## setup common func - for consistency sake follow Eko

In [37]:
class AbstractBlack76Model:
    """
    A base class used to model Black-Scholes option model
    ...
    Parameters
    ----------
    F : float
        The forward price of the underlying asset
    K : float
        The strike price of the options
    discount_factor : float
        The "numeraire" discount factor of the model (i.e. PVBP, compounded discount factor)
    sigma : float
        Volatility
    T : float
        Maturity period (years)
    """

    def __init__(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
    ):
        self.F = F
        self.K = K
        self.sigma = sigma
        self.T = T

        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()
        self.discount_factor = discount_factor

    def _calculate_d1(self) -> float:
        return (np.log(self.F / self.K) + self.sigma**2 / 2 * self.T) / (
            self.sigma * np.sqrt(self.T)
        )

    def _calculate_d2(self) -> float:
        return self.d1 - self.sigma * np.sqrt(self.T)


class VanillaBlack76Model(AbstractBlack76Model):
    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.F * norm.cdf(self.d1) - self.K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            -self.F * norm.cdf(-self.d1) + self.K * norm.cdf(-self.d2)
        )


class AbstractDisplacedDiffusionModel:
    """
    Displaced diffusion is extension of Black76 with an additional parameter beta
    ...
    Parameters
    ----------
    F : float
        The forward price of the underlying asset
    K : float
        The strike price of the options
    discount_factor : float
        The "numeraire" discount factor of the model (i.e. PVBP, compounded discount factor)
    sigma : float
        Volatility
    T : float
        Maturity period (years)
    beta : float
        Displaced diffusion model parameter (0,1], but lecture notes say [0,1]
        https://ink.library.smu.edu.sg/cgi/viewcontent.cgi?article=6976&context=lkcsb_research
    """

    def __init__(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
        beta: float,
    ):
        self.F = F
        self.K = K
        self.sigma = sigma
        self.T = T
        self.beta = beta

        self.adjusted_F = self.F / self.beta
        self.adjusted_K = self.K + ((1 - self.beta) / self.beta) * self.F
        self.adjusted_sigma = self.sigma * self.beta
        self.discount_factor = discount_factor

        self.d1 = self._calculate_d1()
        self.d2 = self._calculate_d2()

    def _calculate_d1(self) -> float:
        return (
            np.log(self.adjusted_F / self.adjusted_K)
            + 0.5 * self.adjusted_sigma**2 * self.T
        ) / (self.adjusted_sigma * np.sqrt(self.T))

    def _calculate_d2(self) -> float:
        return self.d1 - self.adjusted_sigma * np.sqrt(self.T)


class VanillaDisplacedDiffusionModel(AbstractDisplacedDiffusionModel):
    def calculate_call_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_F * norm.cdf(self.d1) - self.adjusted_K * norm.cdf(self.d2)
        )

    def calculate_put_price(self) -> float:
        return self.discount_factor * (
            self.adjusted_K * norm.cdf(-self.d2) - self.adjusted_F * norm.cdf(-self.d1)
        )


class VanillaOption:
    def black_model(
        self, F: float, K: float, discount_factor: float, sigma: float, T: float
    ) -> VanillaBlack76Model:
        return VanillaBlack76Model(F, K, discount_factor, sigma, T)

    def displaced_diffusion_model(
        self,
        F: float,
        K: float,
        discount_factor: float,
        sigma: float,
        T: float,
        beta: float,
    ) -> AbstractDisplacedDiffusionModel:
        return VanillaDisplacedDiffusionModel(F, K, discount_factor, sigma, T, beta)

In [38]:
def implied_volatility(
    S: float, K: float, r: float, price: float, T: float, options_type: str
) -> float:
    try:
        bs_model = lambda x: B76Model(S, K, r, x, T)
        if options_type.lower() == "payer":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_call_price(), 1e-12, 10.0
            )
        elif options_type.lower() == "receiver":
            implied_vol = brentq(
                lambda x: price - bs_model(x).calculate_put_price(), 1e-12, 10.0
            )
        else:
            raise NameError("Payoff type not recognized")
    except Exception:
        implied_vol = np.nan

    return implied_vol

## data description
Lognormal Implied Volatility for IR Swaptions

Strike (Forward + basis point)

In [39]:
swaption_data

,expiry,tenor,-200bps,-150bps,-100bps,-50bps,-25bps,atm,+25bps,+50bps,+100bps,+150bps,+200bps
0,1.0,1.0,0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1.0,2.0,0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
2,1.0,3.0,0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
3,1.0,5.0,0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
4,1.0,10.0,0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355
5,5.0,1.0,0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,0.25980
6,5.0,2.0,0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,0.26760
7,5.0,3.0,0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,0.26150
8,5.0,5.0,0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,0.23400
9,5.0,10.0,0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,0.22250


In [40]:
strike_deltas = (
    swaption_data.columns[2:]
    .str.replace("bps", "")
    .str.replace("atm", "0")
    .str.replace("+", "")
    .to_numpy()
    .astype(float)
)
strike_deltas = strike_deltas / 1e4
strike_deltas

C:\Users\kilo\AppData\Local\Temp\ipykernel_14508\473921626.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  .str.replace("+", "")


array([-0.02  , -0.015 , -0.01  , -0.005 , -0.0025,  0.    ,  0.0025,
        0.005 ,  0.01  ,  0.015 ,  0.02  ])

In [41]:
# note snN_0 is ATM strike
df_snN_0["atm_vol"] = swaption_data["atm"]

In [42]:
df_snN_0

,expiry,tenor,pvbp,pv_float,snN_0,atm_vol
0,1.0,1.0,0.994400,0.031828,0.032007,0.2250
1,1.0,2.0,1.985294,0.066029,0.033259,0.2872
2,1.0,3.0,2.972386,0.101093,0.034011,0.2978
3,1.0,5.0,4.934070,0.173953,0.035255,0.2607
4,1.0,10.0,9.747887,0.374591,0.038428,0.2447
5,5.0,1.0,0.978517,0.038430,0.039274,0.2726
6,5.0,2.0,1.952145,0.078232,0.040075,0.2983
7,5.0,3.0,2.920444,0.117029,0.040072,0.2998
8,5.0,5.0,4.840612,0.198916,0.041093,0.2660
9,5.0,10.0,9.542492,0.416374,0.043634,0.2451


### calibrate dd

In [43]:
vanilla_option = VanillaOption()


def implied_volatility(
    F: float,
    K: float,
    discount_factor: float,
    price: float,
    T: float,
    options_type: str,
) -> float:
    try:
        b76_model = lambda x: vanilla_option.black_model(F, K, discount_factor, x, T)
        if options_type.lower() == "payer":
            implied_vol = brentq(
                lambda x: price - b76_model(x).calculate_call_price(), 1e-12, 10.0
            )
        elif options_type.lower() == "receiver":
            implied_vol = brentq(
                lambda x: price - b76_model(x).calculate_put_price(), 1e-12, 10.0
            )
        else:
            raise NameError("Payoff type not recognized")
    except Exception:
        implied_vol = 0

    return implied_vol

In [44]:
def calculate_displaced_diffusion_price(
    F: float,
    strike: float,
    discount_factor: float,
    sigma: float,
    T: float,
    beta,
    options_type: str,
):
    if options_type == "payer":
        price = vanilla_option.displaced_diffusion_model(
            F, strike, discount_factor, sigma, T, beta
        ).calculate_call_price()
    else:
        price = vanilla_option.displaced_diffusion_model(
            F, strike, discount_factor, sigma, T, beta
        ).calculate_put_price()
    return price


def calculate_displaced_diffusion_implied_vol(
    F: float,
    strike: float,
    discount_factor: float,
    sigma: float,
    T: float,
    beta,
    options_type: str,
) -> float:
    price = calculate_displaced_diffusion_price(
        F, strike, discount_factor, sigma, T, beta, options_type
    )
    implied_vol = implied_volatility(F, strike, discount_factor, price, T, options_type)
    return implied_vol


def calculate_DD_vol_err(x, strikes, vols, F, discount_factor, sigma, T, options_type):
    err = 0.0
    for i, vol in enumerate(vols):
        implied_vol = calculate_displaced_diffusion_implied_vol(
            F, strikes[i], discount_factor, sigma, T, x[0], options_type[i]
        )

        err += (vol - implied_vol) ** 2
    return err

In [45]:
df_snN_0

,expiry,tenor,pvbp,pv_float,snN_0,atm_vol
0,1.0,1.0,0.994400,0.031828,0.032007,0.2250
1,1.0,2.0,1.985294,0.066029,0.033259,0.2872
2,1.0,3.0,2.972386,0.101093,0.034011,0.2978
3,1.0,5.0,4.934070,0.173953,0.035255,0.2607
4,1.0,10.0,9.747887,0.374591,0.038428,0.2447
5,5.0,1.0,0.978517,0.038430,0.039274,0.2726
6,5.0,2.0,1.952145,0.078232,0.040075,0.2983
7,5.0,3.0,2.920444,0.117029,0.040072,0.2998
8,5.0,5.0,4.840612,0.198916,0.041093,0.2660
9,5.0,10.0,9.542492,0.416374,0.043634,0.2451


In [46]:
swaption_data

,expiry,tenor,-200bps,-150bps,-100bps,-50bps,-25bps,atm,+25bps,+50bps,+100bps,+150bps,+200bps
0,1.0,1.0,0.91570,0.62030,0.44130,0.31224,0.26182,0.2250,0.2096,0.2140,0.2434,0.27488,0.30297
1,1.0,2.0,0.83270,0.61240,0.46570,0.35807,0.31712,0.2872,0.2712,0.2684,0.2851,0.31025,0.33523
2,1.0,3.0,0.73920,0.56870,0.44770,0.35745,0.32317,0.2978,0.2829,0.2780,0.2877,0.30725,0.32833
3,1.0,5.0,0.55190,0.44640,0.36510,0.30242,0.27851,0.2607,0.2498,0.2456,0.2512,0.26536,0.28165
4,1.0,10.0,0.41180,0.35040,0.30207,0.26619,0.25351,0.2447,0.2398,0.2382,0.2425,0.25204,0.26355
5,5.0,1.0,0.67800,0.49090,0.38400,0.31485,0.29060,0.2726,0.2604,0.2532,0.2494,0.25320,0.25980
6,5.0,2.0,0.57880,0.46410,0.39033,0.33653,0.31531,0.2983,0.2856,0.2765,0.2671,0.26540,0.26760
7,5.0,3.0,0.53430,0.44440,0.38180,0.33437,0.31536,0.2998,0.2876,0.2782,0.2667,0.26200,0.26150
8,5.0,5.0,0.41990,0.36524,0.32326,0.29005,0.27677,0.2660,0.2573,0.2502,0.2406,0.23570,0.23400
9,5.0,10.0,0.34417,0.30948,0.28148,0.25954,0.25136,0.2451,0.2399,0.2356,0.2291,0.22490,0.22250


In [47]:
ddm_parameters = {}
swaption_calibrated_ddm_vols = {}

for _, row in df_snN_0.iterrows():
    pvbp = row["pvbp"]  # pass pvbp for a given A X B
    expiry = row["expiry"]  # expiry
    tenor = row["tenor"]  # pass tenor for a given A X B; i.e B
    F = row["snN_0"]  # pass SnN(0) for a given A X B; i.e B
    sigma = row["atm_vol"]  # pass SnN(0) ATM Sigma

    # dataframe for strikes idk maybe neater
    df_strike_vol = pd.DataFrame()
    strikes = F + strike_deltas
    filter_condition = (swaption_data["expiry"] == row["expiry"]) & (
        swaption_data["tenor"] == row["tenor"]
    )
    # ugly yes whatever
    vols = swaption_data[filter_condition].T.to_numpy()[2:].T.flatten()
    options_type = np.where(strikes >= F, "payer", "receiver")
    print(f"the strikes {strikes}")
    print(f"the vols {vols}")
    print(f"the F {F}")
    print(f"the discount_factor {pvbp}")
    print(f"the sigma {sigma}")
    print(f"the expiry {expiry}")
    print(f"the options_types {options_type}")
    initial_guess = [0.96]  # beta
    res = least_squares(
        lambda x: calculate_DD_vol_err(
            x,
            strikes,
            vols,
            F,
            pvbp,
            sigma,
            expiry,
            options_type,
        ),
        initial_guess,
        bounds=(0, 1),
    )
    beta = res.x[0]
    # reuse eko for pivot table
    key = "{}Yx{}Y".format(expiry, tenor)
    ddm_parameters[key] = {
        "expiry": expiry,
        "tenor": tenor,
        "sigma": sigma,
        "beta": beta,
    }

the strikes [0.012007 0.017007 0.022007 0.027007 0.029507 0.032007 0.034507 0.037007
 0.042007 0.047007 0.052007]
the vols [0.9157  0.6203  0.4413  0.31224 0.26182 0.225   0.2096  0.214   0.2434
 0.27488 0.30297]
the F 0.0320069991937646
the discount_factor 0.9944003027051374
the sigma 0.225
the expiry 1.0
the options_types ['receiver' 'receiver' 'receiver' 'receiver' 'receiver' 'payer' 'payer'
 'payer' 'payer' 'payer' 'payer']


the strikes [0.01325923 0.01825923 0.02325923 0.02825923 0.03075923 0.03325923
 0.03575923 0.03825923 0.04325923 0.04825923 0.05325923]
the vols [0.8327  0.6124  0.4657  0.35807 0.31712 0.2872  0.2712  0.2684  0.2851
 0.31025 0.33523]
the F 0.0332592257741675
the discount_factor 1.9852943450971712
the sigma 0.2872
the expiry 1.0
the options_types ['receiver' 'receiver' 'receiver' 'receiver' 'receiver' 'payer' 'payer'
 'payer' 'payer' 'payer' 'payer']
the strikes [0.01401073 0.01901073 0.02401073 0.02901073 0.03151073 0.03401073
 0.03651073 0.03901073 0.04401073 0.04901073 0.05401073]
the vols [0.7392  0.5687  0.4477  0.35745 0.32317 0.2978  0.2829  0.278   0.2877
 0.30725 0.32833]
the F 0.0340107263527209
the discount_factor 2.9723856176881838
the sigma 0.2978
the expiry 1.0
the options_types ['receiver' 'receiver' 'receiver' 'receiver' 'receiver' 'payer' 'payer'
 'payer' 'payer' 'payer' 'payer']
the strikes [0.01525547 0.02025547 0.02525547 0.03025547 0.03275547 0.03525547
 0.03775547

In [48]:
print(ddm_parameters.keys())

dict_keys(['1.0Yx1.0Y', '1.0Yx2.0Y', '1.0Yx3.0Y', '1.0Yx5.0Y', '1.0Yx10.0Y', '5.0Yx1.0Y', '5.0Yx2.0Y', '5.0Yx3.0Y', '5.0Yx5.0Y', '5.0Yx10.0Y', '10.0Yx1.0Y', '10.0Yx2.0Y', '10.0Yx3.0Y', '10.0Yx5.0Y', '10.0Yx10.0Y'])


In [49]:
df_ddm_parameters = pd.DataFrame(ddm_parameters.values(), index=ddm_parameters.keys())
df_ddm_parameters.index.name = "Swaption"
df_ddm_sigma = df_ddm_parameters.pivot(index="expiry", columns="tenor", values="sigma")
df_ddm_beta = df_ddm_parameters.pivot(index="expiry", columns="tenor", values="beta")

In [50]:
print("DDM Sigma Parameters:")
df_ddm_sigma

DDM Sigma Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,0.2250,0.2872,0.2978,0.2607,0.2447
5.0,0.2726,0.2983,0.2998,0.2660,0.2451
10.0,0.2854,0.2928,0.2940,0.2674,0.2437


In [51]:
print("DDM Beta Parameters:")
df_ddm_beta

DDM Beta Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,1.096224e-07,5.006531e-13,1.347635e-09,7.035170e-08,6.300867e-07
5.0,7.031073e-07,4.022960e-10,5.017088e-07,5.110078e-07,5.412154e-02
10.0,2.427870e-09,3.411163e-06,5.238218e-08,7.918388e-06,1.972126e-04


### calibrate SABR

In [52]:
# retry
def SABR_model(F, K, T, alpha, beta, rho, nu):
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta) ** 2) / 24) * alpha * alpha / (F ** (2 - 2 * beta))
        numer2 = 0.25 * rho * beta * nu * alpha / (F ** (1 - beta))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        VolAtm = alpha * (1 + (numer1 + numer2 + numer3) * T) / (F ** (1 - beta))
        sabrsigma = VolAtm
    else:
        z = (nu / alpha) * ((F * X) ** (0.5 * (1 - beta))) * np.log(F / X)
        zhi = np.log((((1 - 2 * rho * z + z * z) ** 0.5) + z - rho) / (1 - rho))
        numer1 = (((1 - beta) ** 2) / 24) * ((alpha * alpha) / ((F * X) ** (1 - beta)))
        numer2 = 0.25 * rho * beta * nu * alpha / ((F * X) ** ((1 - beta) / 2))
        numer3 = ((2 - 3 * rho * rho) / 24) * nu * nu
        numer = alpha * (1 + (numer1 + numer2 + numer3) * T) * z
        denom1 = ((1 - beta) ** 2 / 24) * (np.log(F / X)) ** 2
        denom2 = (((1 - beta) ** 4) / 1920) * ((np.log(F / X)) ** 4)
        denom = ((F * X) ** ((1 - beta) / 2)) * (1 + denom1 + denom2) * zhi
        sabrsigma = numer / denom

    return sabrsigma


def calculate_SABR_vol_err(x, strikes, vols, F, T, beta):
    err = 0.0
    for i, vol in enumerate(vols):
        err += (vol - SABR_model(F, strikes[i], T, x[0], beta, x[1], x[2])) ** 2
    return err

In [53]:
beta = 0.9
sabr_parameters = {}

for _, row in df_snN_0.iterrows():
    # pvbp = row["pvbp"]  # pass pvbp for a given A X B
    expiry = row["expiry"]  # expiry
    tenor = row["tenor"]  # pass tenor for a given A X B; i.e B
    F = row["snN_0"]  # pass SnN(0) for a given A X B; i.e B
    sigma = row["atm_vol"]  # pass SnN(0) ATM Sigma

    # dataframe for strikes idk maybe neater
    df_strike_vol = pd.DataFrame()
    strikes = F + strike_deltas
    filter_condition = (swaption_data["expiry"] == row["expiry"]) & (
        swaption_data["tenor"] == row["tenor"]
    )
    vols = swaption_data[filter_condition].T.to_numpy()[2:]
    # construct the dataframe
    df_strike_vol["strikes"] = strikes
    df_strike_vol["vols"] = vols
    df_strike_vol["options_type"] = np.where(
        df_strike_vol["strikes"] >= F, "payer", "receiver"
    )
    print(f"the strikes {df_strike_vol['strikes']}")
    print(f"the vols {df_strike_vol['vols']}")
    print(f"the F {F}")
    print(f"the discount_factor {pvbp}")
    print(f"the sigma {sigma}")
    print(f"the tenor {tenor}")
    print(f"the options_types {df_strike_vol['options_type']}")
    print("=====================================================")
    initial_guess = [0.18, -0.45, 0.5]
    res = least_squares(
        lambda x: calculate_SABR_vol_err(
            x,
            strikes,
            vols,
            F,
            expiry,
            beta,
        ),
        initial_guess,
        max_nfev=6000,
    )

    alpha, rho, nu = res.x
    # reuse eko for pivot table
    key = "{}Yx{}Y".format(expiry, tenor)
    sabr_parameters[key] = {
        "expiry": expiry,
        "tenor": tenor,
        "alpha": alpha,
        "beta": beta,
        "rho": rho,
        "nu": nu,
    }

the strikes 0     0.012007
1     0.017007
2     0.022007
3     0.027007
4     0.029507
5     0.032007
6     0.034507
7     0.037007
8     0.042007
9     0.047007
10    0.052007
Name: strikes, dtype: float64
the vols 0     0.91570
1     0.62030
2     0.44130
3     0.31224
4     0.26182
5     0.22500
6     0.20960
7     0.21400
8     0.24340
9     0.27488
10    0.30297
Name: vols, dtype: float64
the F 0.0320069991937646
the discount_factor 9.264214201574296
the sigma 0.225
the tenor 1.0
the options_types 0     receiver
1     receiver
2     receiver
3     receiver
4     receiver
5        payer
6        payer
7        payer
8        payer
9        payer
10       payer
Name: options_type, dtype: object


the strikes 0     0.013259
1     0.018259
2     0.023259
3     0.028259
4     0.030759
5     0.033259
6     0.035759
7     0.038259
8     0.043259
9     0.048259
10    0.053259
Name: strikes, dtype: float64
the vols 0     0.83270
1     0.61240
2     0.46570
3     0.35807
4     0.31712
5     0.28720
6     0.27120
7     0.26840
8     0.28510
9     0.31025
10    0.33523
Name: vols, dtype: float64
the F 0.0332592257741675
the discount_factor 9.264214201574296
the sigma 0.2872
the tenor 2.0
the options_types 0     receiver
1     receiver
2     receiver
3     receiver
4     receiver
5        payer
6        payer
7        payer
8        payer
9        payer
10       payer
Name: options_type, dtype: object
the strikes 0     0.014011
1     0.019011
2     0.024011
3     0.029011
4     0.031511
5     0.034011
6     0.036511
7     0.039011
8     0.044011
9     0.049011
10    0.054011
Name: strikes, dtype: float64
the vols 0     0.73920
1     0.56870
2     0.44770
3     0.35745
4     0.32317
5     

In [54]:
df_sabr_parameters = pd.DataFrame(
    sabr_parameters.values(), index=sabr_parameters.keys()
)
df_sabr_parameters.index.name = "Swaption"
df_sabr_alpha = df_sabr_parameters.pivot(
    index="expiry", columns="tenor", values="alpha"
)
df_sabr_beta = df_sabr_parameters.pivot(index="expiry", columns="tenor", values="beta")
df_sabr_rho = df_sabr_parameters.pivot(index="expiry", columns="tenor", values="rho")
df_sabr_nu = df_sabr_parameters.pivot(index="expiry", columns="tenor", values="nu")

In [55]:
print("SABR Alpha Parameters:")
df_sabr_alpha

SABR Alpha Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,0.139074,0.184650,0.196852,0.178042,0.171143
5.0,0.166618,0.199534,0.210305,0.190972,0.177051
10.0,0.178247,0.196184,0.207769,0.199167,0.176347


In [56]:
print("SABR Beta Parameters:")
df_sabr_beta

SABR Beta Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,0.9,0.9,0.9,0.9,0.9
5.0,0.9,0.9,0.9,0.9,0.9
10.0,0.9,0.9,0.9,0.9,0.9


In [57]:
print("SABR Rho Parameters:")
df_sabr_rho

SABR Rho Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,-0.633223,-0.525114,-0.482848,-0.414420,-0.265908
5.0,-0.585668,-0.547028,-0.549583,-0.511010,-0.438524
10.0,-0.548309,-0.547143,-0.552303,-0.556152,-0.486600


In [58]:
print("SABR Nu Parameters:")
df_sabr_nu

SABR Nu Parameters:


tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,2.049416,1.677383,1.438120,1.064975,0.777278
5.0,1.340427,1.062004,0.936601,0.672014,0.497465
10.0,1.010141,0.928204,0.870391,0.719435,0.582470


### price swaptions

* payer 2y × 10y K = 1%, 2%, 3%, 4%, 5%, 6%, 7%, 8%
* receiver 8y × 10y K = 1%, 2%, 3%, 4%, 5%, 6%, 7%, 8%


In [59]:
df_question = pd.DataFrame()
df_question["expiry"] = [2.0, 8.0]
df_question["tenor"] = [10.0, 10.0]
df_question["option_type"] = ["payer", "receiver"]
strikes = np.arange(0.01, 0.08, 0.01)

In [60]:
df_question

,expiry,tenor,option_type
0,2.0,10.0,payer
1,8.0,10.0,receiver


In [61]:
for x, row in df_question.iterrows():
    expiry = df_question.loc[x, "expiry"]
    tenor = df_question.loc[x, "tenor"]
    time_sum = expiry + tenor
    rows_to_work = df_combined[
        (df_combined["tenor"] >= expiry) & (df_combined["tenor"] <= time_sum)
    ]
    # drop first row because the first payment takes place like 6m after the expiry ends
    # e.g. if expiry = 1y, then first payment = 1.5y
    rows_to_work = rows_to_work.drop(rows_to_work.index[[0]])
    df_question.loc[x, "pv_fix_nok"] = 0.5 * rows_to_work["ois_df"].sum()
    df_question.loc[x, "pv_float"] = (
        0.5 * (rows_to_work["ois_df"] * rows_to_work["fw_libor"]).sum()
    )
    df_question.loc[x, "k_rate"] = (
        df_question.loc[x, "pv_float"] / df_question.loc[x, "pv_fix_nok"]
    )
    # print(rows_to_work)

In [62]:
df_question = df_question.loc[df_question.index.repeat(8)].reset_index(drop=True)
df_question["strikes"] = np.tile(strikes, 2)

In [63]:
print("working df for pricing swaptions")
df_question

working df for pricing swaptions


,expiry,tenor,option_type,pv_fix_nok,pv_float,k_rate,strikes
0,2.0,10.0,payer,9.699536,0.384435,0.039634,0.01
1,2.0,10.0,payer,9.699536,0.384435,0.039634,0.02
2,2.0,10.0,payer,9.699536,0.384435,0.039634,0.03
3,2.0,10.0,payer,9.699536,0.384435,0.039634,0.04
4,2.0,10.0,payer,9.699536,0.384435,0.039634,0.05
5,2.0,10.0,payer,9.699536,0.384435,0.039634,0.06
6,2.0,10.0,payer,9.699536,0.384435,0.039634,0.07
7,2.0,10.0,payer,9.699536,0.384435,0.039634,0.08
8,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.01
9,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.02


In [64]:
df_ddm_beta

tenor,1.0,2.0,3.0,5.0,10.0
expiry,,,,,
1.0,1.096224e-07,5.006531e-13,1.347635e-09,7.035170e-08,6.300867e-07
5.0,7.031073e-07,4.022960e-10,5.017088e-07,5.110078e-07,5.412154e-02
10.0,2.427870e-09,3.411163e-06,5.238218e-08,7.918388e-06,1.972126e-04


In [65]:
# 1d interp
def get_interp_param(mydataframe):

    y_known = mydataframe[10.0].to_numpy()

    x_known = mydataframe.index.to_numpy()

    x_interpolate = [2.0, 8.0]

    y_interpolate = np.interp(x_interpolate, x_known, y_known)

    # interp beta

    interp = dict(zip(x_interpolate, y_interpolate))

    return interp

In [66]:
# DDM
ddm_beta_interp = RegularGridInterpolator(
    (df_ddm_beta.index, df_ddm_beta.columns), df_ddm_beta.values, method="linear"
)
ddm_sigma_interp = RegularGridInterpolator(
    (df_ddm_sigma.index, df_ddm_sigma.columns), df_ddm_sigma.values, method="linear"
)

# SABR
sabr_alpha_interp = RegularGridInterpolator(
    (df_sabr_alpha.index, df_sabr_alpha.columns), df_sabr_alpha.values, method="linear"
)
sabr_beta_interp = RegularGridInterpolator(
    (df_sabr_beta.index, df_sabr_beta.columns), df_sabr_beta.values, method="linear"
)
sabr_rho_interp = RegularGridInterpolator(
    (df_sabr_rho.index, df_sabr_rho.columns), df_sabr_rho.values, method="linear"
)
sabr_nu_interp = RegularGridInterpolator(
    (df_sabr_nu.index, df_sabr_nu.columns), df_sabr_nu.values, method="linear"
)

In [67]:
df_question_ddm = df_question.copy()
for _, row in df_question_ddm.iterrows():
    F, pvbp, expiry, tenor, option_type = (
        df_question_ddm.loc[_, "k_rate"],
        df_question_ddm.loc[_, "pv_fix_nok"],
        df_question_ddm.loc[_, "expiry"],
        df_question_ddm.loc[_, "tenor"],
        df_question_ddm.loc[_, "option_type"],
    )
    # very efficient yes
    point_to_interp = (expiry, tenor)
    sigma = ddm_sigma_interp(point_to_interp)
    beta = ddm_beta_interp(point_to_interp)
    dd_price = calculate_displaced_diffusion_price(
        F,
        df_question_ddm.loc[_, "strikes"],
        pvbp,
        sigma,
        expiry,
        beta,
        option_type,
    )
    df_question_ddm.loc[_, "sigma"] = sigma
    df_question_ddm.loc[_, "beta"] = beta
    df_question_ddm.loc[_, "dd_price"] = dd_price

In [68]:
print("ddm pricing swaptions")
df_question_ddm

ddm pricing swaptions


,expiry,tenor,option_type,pv_fix_nok,pv_float,k_rate,strikes,sigma,beta,dd_price
0,2.0,10.0,payer,9.699536,0.384435,0.039634,0.01,0.24480,0.013531,0.288143
1,2.0,10.0,payer,9.699536,0.384435,0.039634,0.02,0.24480,0.013531,0.194938
2,2.0,10.0,payer,9.699536,0.384435,0.039634,0.03,0.24480,0.013531,0.112328
3,2.0,10.0,payer,9.699536,0.384435,0.039634,0.04,0.24480,0.013531,0.051345
4,2.0,10.0,payer,9.699536,0.384435,0.039634,0.05,0.24480,0.013531,0.017364
5,2.0,10.0,payer,9.699536,0.384435,0.039634,0.06,0.24480,0.013531,0.004104
6,2.0,10.0,payer,9.699536,0.384435,0.039634,0.07,0.24480,0.013531,0.000651
7,2.0,10.0,payer,9.699536,0.384435,0.039634,0.08,0.24480,0.013531,0.000067
8,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.01,0.24426,0.021767,0.019023
9,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.02,0.24426,0.021767,0.033942


In [69]:
df_question_sabr = df_question.copy()
for _, row in df_question_sabr.iterrows():
    F, expiry, tenor, option_type = (
        df_question_sabr.loc[_, "k_rate"],
        df_question_sabr.loc[_, "expiry"],
        df_question_sabr.loc[_, "tenor"],
        df_question_sabr.loc[_, "option_type"],
    )
    # very efficient yes
    point_to_interp = (expiry, tenor)
    alpha = sabr_alpha_interp(point_to_interp)
    beta = sabr_beta_interp(point_to_interp)
    nu = sabr_nu_interp(point_to_interp)
    rho = sabr_rho_interp(point_to_interp)
    sabr_sigma = SABR_model(
        F,
        df_question_sabr.loc[_, "strikes"],
        expiry,
        alpha,
        beta,
        rho,
        nu,
    )
    black_sabr_model = vanilla_option.black_model(
        F,
        df_question_sabr.loc[_, "strikes"],
        df_question_sabr.loc[_, "pv_fix_nok"],
        sabr_sigma,
        expiry,
    )
    sabr_price = (
        black_sabr_model.calculate_call_price()
        if option_type == "payer"
        else black_sabr_model.calculate_put_price()
    )
    df_question_sabr.loc[_, "alpha"] = alpha
    df_question_sabr.loc[_, "beta"] = beta
    df_question_sabr.loc[_, "nu"] = nu
    df_question_sabr.loc[_, "rho"] = rho
    df_question_sabr.loc[_, "sabr_sigma"] = sabr_sigma
    df_question_sabr.loc[_, "sabr_price"] = sabr_price

    # df_question_sabr.loc[_, "black_sabr_F"] = F
    # df_question_sabr.loc[_, "black_sabr_strike"] = df_question_sabr.loc[_, "strikes"]
    # df_question_sabr.loc[_, "black_sabr_disc_factor"] = df_question_sabr.loc[
    #    _, "pv_fix_nok"
    # ]
    # df_question_sabr.loc[_, "black_sabr_sigma"] = sabr_sigma
    # df_question_sabr.loc[_, "black_sabr_T"] = expiry

In [70]:
print("sabr pricing swaptions")
df_question_sabr

sabr pricing swaptions


,expiry,tenor,option_type,pv_fix_nok,pv_float,k_rate,strikes,alpha,beta,nu,rho,sabr_sigma,sabr_price
0,2.0,10.0,payer,9.699536,0.384435,0.039634,0.01,0.172620,0.9,0.707325,-0.309062,0.549456,0.289608
1,2.0,10.0,payer,9.699536,0.384435,0.039634,0.02,0.172620,0.9,0.707325,-0.309062,0.394200,0.198315
2,2.0,10.0,payer,9.699536,0.384435,0.039634,0.03,0.172620,0.9,0.707325,-0.309062,0.301380,0.115198
3,2.0,10.0,payer,9.699536,0.384435,0.039634,0.04,0.172620,0.9,0.707325,-0.309062,0.248701,0.052158
4,2.0,10.0,payer,9.699536,0.384435,0.039634,0.05,0.172620,0.9,0.707325,-0.309062,0.240170,0.021354
5,2.0,10.0,payer,9.699536,0.384435,0.039634,0.06,0.172620,0.9,0.707325,-0.309062,0.256699,0.010715
6,2.0,10.0,payer,9.699536,0.384435,0.039634,0.07,0.172620,0.9,0.707325,-0.309062,0.278507,0.006598
7,2.0,10.0,payer,9.699536,0.384435,0.039634,0.08,0.172620,0.9,0.707325,-0.309062,0.299721,0.004621
8,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.01,0.176629,0.9,0.548468,-0.467370,0.529420,0.019324
9,8.0,10.0,receiver,9.375970,0.456714,0.048711,0.02,0.176629,0.9,0.548468,-0.467370,0.406003,0.038353
